# Lab 1

In [1]:
import pandas as pd

# load in files
april_df = pd.read_csv(r"C:\Users\achur\OneDrive\Desktop\School\CP Spring 2024\545\GSB545\Labs\April sleep data - Sheet1.csv")
december_df = pd.read_csv(r"C:\Users\achur\OneDrive\Desktop\School\CP Spring 2024\545\GSB545\Labs\December Sleep data - Sheet1.csv")
february_df = pd.read_csv(r"C:\Users\achur\OneDrive\Desktop\School\CP Spring 2024\545\GSB545\Labs\February sleep data - Sheet1 (1).csv")
january_df = pd.read_csv(r"C:\Users\achur\OneDrive\Desktop\School\CP Spring 2024\545\GSB545\Labs\January sleep data - Sheet1.csv")
march_df = pd.read_csv(r"C:\Users\achur\OneDrive\Desktop\School\CP Spring 2024\545\GSB545\Labs\March sleep data - Sheet1.csv")
november_df = pd.read_csv(r"C:\Users\achur\OneDrive\Desktop\School\CP Spring 2024\545\GSB545\Labs\November Sleep Data - Sheet1.csv")